In [2]:
import pandas as pd
import numpy as np
import yaml
import logging
from pathlib import Path
from time import sleep

from utils import nfl_weeks_pull, game_keys_pull
from yahoo_query import league_season_data

dates = [
    # np.datetime64("2021-09-28"),
    np.datetime64("2020-09-28"),
    np.datetime64("2019-09-28"),
    np.datetime64("2018-09-28"),
    np.datetime64("2017-09-28"),
    np.datetime64("2016-09-28"),
    np.datetime64("2015-09-28"),
    np.datetime64("2014-09-28"),
    np.datetime64("2013-09-28"),
    np.datetime64("2012-09-28"),
]

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
NFL_WEEKS = nfl_weeks_pull()
GAME_KEYS = game_keys_pull(first="no")

for today in dates:

    SEASON = today.astype("datetime64[Y]").item().year
    LEAGUE_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["league_ID"].values[0]
    GAME_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["game_id"].values[0]
    nfl_weeks_list = list(NFL_WEEKS["week"][NFL_WEEKS["game_id"] == GAME_ID])

    try:
        with open(PATH) as file:
            credentials = yaml.load(file, Loader=yaml.FullLoader)

    except Exception as error:
        print(error)

    CONSUMER_KEY = credentials["YFPY_CONSUMER_KEY"]
    CONSUMER_SECRET = credentials["YFPY_CONSUMER_SECRET"]

    league = league_season_data(
        auth_dir=PATH.parent,
        league_id=LEAGUE_ID,
        game_id=GAME_ID,
        game_code="nfl",
        offline=False,
        all_output_as_json=False,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        browser_callback=True,
    )

    league.metadata(first_time="no")

    # if int(SEASON) == 2021:
    #     league.all_game_keys()
    #     sleep(0.1)
    #     league.all_nfl_weeks()
    #     sleep(0.1)

    #     league.metadata(first_time="yes")
    #     sleep(0.1)

    #     league.set_roster_pos_stat_cat(first_time="yes")
    #     sleep(0.1)

    #     league.draft_results(first_time="yes")
    #     sleep(0.1)

    #     league.teams_and_standings(first_time="yes")
    #     sleep(0.1)

    #     # league.players_list(first_time="yes")

    #     league.matchups_by_week_regseason(first_time="yes", nfl_week=1)
    #     for week in nfl_weeks_list:
    #         league.matchups_by_week_regseason(first_time="no", nfl_week=week)
    #         sleep(0.1)

    #     league.team_roster_by_week(first_time="yes", nfl_week=1)
    #     for week in nfl_weeks_list:
    #         league.team_roster_by_week(first_time="no", nfl_week=week)
    #         sleep(0.1)

    #     league.team_points_by_week(first_time="yes", nfl_week=1)
    #     for week in nfl_weeks_list:
    #         league.team_points_by_week(first_time="no", nfl_week=week)
    #         sleep(0.1)

    # else:
    #     league.metadata(first_time="no")
    #     sleep(0.1)

    #     league.set_roster_pos_stat_cat(first_time="no")
    #     sleep(0.1)

    #     league.draft_results(first_time="no")
    #     sleep(0.1)

    #     league.teams_and_standings(first_time="no")
    #     sleep(0.1)

    #     for week in nfl_weeks_list:
    #         league.matchups_by_week_regseason(first_time="no", nfl_week=week)
    #         sleep(0.1)

    #     # league.players_list(first_time="no")

    #     for week in nfl_weeks_list:
    #         league.team_roster_by_week(first_time="no", nfl_week=week)
    #         sleep(0.1)

    #     for week in nfl_weeks_list:
    #         league.team_points_by_week(first_time="no", nfl_week=week)
    #         sleep(0.1)

[2022-07-28 20:10:41,165 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7efdd3b24180; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=prod'', closed: 0>
DEBUG:sqlalchemy.pool.impl.QueuePool:Created new connection <connection object at 0x7efdd3b24180; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=prod'', closed: 0>
[2022-07-28 20:10:41,168 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
INFO:sqlalchemy.engine.Engine:select pg_catalog.version()
[2022-07-28 20:10:41,169 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
INFO:sqlalchemy.engine.Engine:[raw sql] {}
[2022-07-28 20:10:41,170 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
DEBUG:sqlalchemy.engine.Engine:Col ('version',)
[2022-07-28 20:10:41,171 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 

Successfully pulled: SELECT * FROM prod.nfl_weeks
Successfully pulled: SELECT * FROM prod.game_keys


[2022-07-28 20:10:41,783 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7efdd3b24680; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
DEBUG:sqlalchemy.pool.impl.QueuePool:Created new connection <connection object at 0x7efdd3b24680; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
[2022-07-28 20:10:41,785 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
INFO:sqlalchemy.engine.Engine:select pg_catalog.version()
[2022-07-28 20:10:41,786 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
INFO:sqlalchemy.engine.Engine:[raw sql] {}
[2022-07-28 20:10:41,787 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
DEBUG:sqlalchemy.engine.Engine:Col ('version',)
[2022-07-28 20:10:41,788 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14

Successfully pulled: SELECT * FROM dev.league_settings
Error extra data after last expected column
CONTEXT:  COPY league_metadata, line 1: "yahoo,90.0,1631059200.0,live,406,0.0,t,0.0,103661,10.0,4.0,4.0,1.0,ALL,16.0,W,head,2021-11-27,vote,1..."

Upload unsuccessful: league_metadata


KeyboardInterrupt: 

In [ ]:
# from db_psql_model import DatabaseCursor
# query = "SELECT * from dev.weeklyteampoints"
# drop_dupes = DatabaseCursor(PATH, options="-c search_path=dev").copy_data_from_postgres(query)
# print(drop_dupes.shape)
# drop_dupes.drop_duplicates(ignore_index=True, inplace=True)
# print(drop_dupes.shape)
# DatabaseCursor(PATH, options="-c search_path=dev").copy_table_to_postgres_new(drop_dupes, "weeklyteampoints", first_time="yes")